In [1]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
import cftime
import datetime
from datetime import date
from matplotlib import pyplot
from matplotlib import colors
from matplotlib import font_manager
from matplotlib.cm import ScalarMappable
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib.ticker as mticker
import numpy
import pandas
from PIL import Image
import random
import xarray as xr

In [2]:
Diri = '/glade/u/home/whimkao//ExtraTrack/ExtraTrack_Data/Output_Files_V6/'
NetCDF_Diri = '/glade/derecho/scratch/whimkao/NetCDF_Files_V6/'
Output_Diri = '/glade/u/home/whimkao//ExtraTrack/ExtraTrack_Github/RCP_Figs/Analysis_Figs_V6.6.3/'

In [3]:
# Open File
def Open_File(File):
    DF = pandas.read_csv(File)
    DF = DF.drop("Unnamed: 0", axis=1)
    return (DF)

In [4]:
# Open Each File
def Files_Open(Model, Diri, Subset):
    Data_DF = Open_File(Diri+Model+'_Data_'+Subset+'_Output_V6.csv')
    ET_DF = Open_File(Diri+Model+'_ET_'+Subset+'_Output_V6.csv')
    Codes_DF = Open_File(Diri+Model+'_Codes_Output_V6.csv')
# Edit Time Format
    Time, Begin_Time, Compl_Time, Trop_Peak_Time, Peak_Time, Birth_Time = [], [], [], [], [], []
    for i in range(len(Data_DF)):
        Time.append(Datetime(Data_DF["Time(Z)"][i]))
    for j in range(len(ET_DF)):
        Begin_Time.append(Datetime(ET_DF["ET Begin Time"][j]))
        Compl_Time.append(Datetime(ET_DF["ET Complete Time"][j]))
        Trop_Peak_Time.append(Datetime(ET_DF["Trop Peak Time"][j]))
        Peak_Time.append(Datetime(ET_DF["Peak Time"][j]))
        Birth_Time.append(Datetime(ET_DF["Genesis Time"][j]))
    Data_DF["Time(Z)"] = Time
    ET_DF["ET Begin Time"] = Begin_Time
    ET_DF["ET Complete Time"] = Compl_Time
    ET_DF["Trop Peak Time"] = Trop_Peak_Time
    ET_DF["Peak Time"] = Peak_Time
    ET_DF["Genesis Time"] = Birth_Time
    return (Data_DF, ET_DF, Codes_DF)

In [5]:
def Datetime(Time):
    try:
        New_Time = datetime.datetime.strptime(Time, '%Y-%m-%d %H:%M:%S')
    except:
        New_Time = numpy.nan
    return (New_Time)

In [6]:
# Create Phase Distribution Bins
def Create_Bins(Min, Max, Bin_Width):
    Bins = numpy.arange(Min, Max+Bin_Width, Bin_Width)
    return (Bins)
Lat_Bins = Create_Bins(10,60,5)
Lon_Bins = Create_Bins(-100,20,5)

In [7]:
Num_Years = numpy.array([90,93,93])

In [8]:
# Function to Find Distance Between Two Points
def Find_Distance(y1, y2, x1, x2):
    Start_Lat = y1 * numpy.pi / 180
    End_Lat = y2 * numpy.pi / 180
    Start_Lon = x1 * numpy.pi / 180
    End_Lon = x2 * numpy.pi / 180
    Lat_Diff = End_Lat - Start_Lat
    Lon_Diff = End_Lon - Start_Lon
    Earth_Rad = 6378
    Distance = 2 * Earth_Rad * numpy.sqrt((numpy.sin(Lat_Diff/2))**2 + \
    numpy.cos(Start_Lat) * numpy.cos(End_Lat) * (numpy.sin(Lon_Diff/2))**2)
    return (Distance)

In [9]:
# Find a Specific Storm Within the DataFrame
def Find_Storm(DF, Code):
    DF_Storm = DF[DF["Code"] == Code].reset_index()
    return (DF_Storm)

In [10]:
Control_Data, Control_ET, Control_Codes = Files_Open("Control", Diri, "SubsetB")
RCP45_Data, RCP45_ET, RCP45_Codes = Files_Open("RCP45", Diri, "SubsetB")
RCP85_Data, RCP85_ET, RCP85_Codes = Files_Open("RCP85", Diri, "SubsetB")

In [11]:
Months_Name = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

In [12]:
# Create Function to Open Monthly Files
def Open_Monthly_File(Model):
    DS_Dict = {}
    for i in range(12):
        File_Name = str(Model+'_Output_'+Months_Name[i]+'_V6.nc')
        Month_DS = xr.open_dataset(NetCDF_Diri + File_Name)
        DS_Dict[Months_Name[i]] = Month_DS
        print (i)
    return (DS_Dict)

In [13]:
# Open Monthly Files
Control_DS_Dict = Open_Monthly_File("Control")
RCP45_DS_Dict = Open_Monthly_File("RCP45")
RCP85_DS_Dict = Open_Monthly_File("RCP85")

0
1
2
3
4
5
6
7
8
9
10
11
0
1
2
3
4
5
6
7
8
9
10
11
0
1
2
3
4
5
6
7
8
9
10
11


In [14]:
Control_DS_Dict['Sep']

<xarray.Dataset>
Dimensions:  (time: 90, plev: 13, lat: 61, lon: 101)
Coordinates:
  * lat      (lat) float64 0.0 1.0 2.0 3.0 4.0 5.0 ... 56.0 57.0 58.0 59.0 60.0
  * lon      (lon) float64 -100.0 -99.0 -98.0 -97.0 -96.0 ... -3.0 -2.0 -1.0 0.0
  * plev     (plev) float64 1e+05 9.25e+04 8.5e+04 7e+04 ... 1.5e+04 1e+04 5e+03
  * time     (time) datetime64[ns] 1985-09-15 1986-09-15 ... 2014-09-27
Data variables:
    T        (time, plev, lat, lon) float32 ...
    TS       (time, lat, lon) float32 ...
    Z3       (time, plev, lat, lon) float32 ...
    PSL      (time, lat, lon) float32 ...
    U        (time, plev, lat, lon) float32 ...
    V        (time, plev, lat, lon) float32 ...
    FLUT     (time, lat, lon) float32 ...
    TMQ      (time, lat, lon) float32 ...
    OMEGA    (time, plev, lat, lon) float32 ...
    OCNFRAC  (time, lat, lon) float32 ...
Attributes: (12/21)
    ne:                         0
    np:                         4
    Conventions:                CF-1.0
    source:                     CAM
    title:                      Regridded version of tmp.nc
    Version:                    $Name$
    ...                         ...
    history_of_appended_files:  Sun Apr 13 09:20:41 2025: Appended file /glad...
    remap_script:               ncremap
    remap_hostname:             casper-login1
    remap_version:              5.3.1
    map_file:                   /glade/work/zarzycki/maps/hyperion/map_ne0np4...
    input_file:                 /glade/u/home/zarzycki/scratch/hyperion/CHEY....

In [15]:
# Combine Monthly Files Into Seasons
def Combine_Monthly_Files(DS_Dict):
    DS_Dict['Mar-May'] = xr.concat([DS_Dict['Mar'], DS_Dict['Apr'], DS_Dict['May']], dim='time')
    DS_Dict['Jun-Aug'] = xr.concat([DS_Dict['Jun'], DS_Dict['Jul'], DS_Dict['Aug']], dim='time')
    DS_Dict['Sep-Nov'] = xr.concat([DS_Dict['Sep'], DS_Dict['Oct'], DS_Dict['Nov']], dim='time')
    DS_Dict['Dec-Feb'] = xr.concat([DS_Dict['Dec'], DS_Dict['Jan'], DS_Dict['Feb']], dim='time')
    DS_Dict['Dec-May'] = xr.concat([DS_Dict['Dec-Feb'], DS_Dict['Mar-May']], dim='time')
    DS_Dict['Annual'] = xr.concat([DS_Dict['Dec-May'], DS_Dict['Jun-Aug'], DS_Dict['Sep-Nov'], DS_Dict['Dec-Feb']], dim='time')
    return (DS_Dict)

In [16]:
Control_DS_Dict = Combine_Monthly_Files(Control_DS_Dict)
RCP45_DS_Dict = Combine_Monthly_Files(RCP45_DS_Dict)
RCP85_DS_Dict = Combine_Monthly_Files(RCP85_DS_Dict)

In [17]:
Control_DS_Dict['Sep-Nov']

<xarray.Dataset>
Dimensions:  (time: 270, plev: 13, lat: 61, lon: 101)
Coordinates:
  * lat      (lat) float64 0.0 1.0 2.0 3.0 4.0 5.0 ... 56.0 57.0 58.0 59.0 60.0
  * lon      (lon) float64 -100.0 -99.0 -98.0 -97.0 -96.0 ... -3.0 -2.0 -1.0 0.0
  * plev     (plev) float64 1e+05 9.25e+04 8.5e+04 7e+04 ... 1.5e+04 1e+04 5e+03
  * time     (time) datetime64[ns] 1985-09-15 1986-09-15 ... 2014-11-27
Data variables:
    T        (time, plev, lat, lon) float32 292.8 292.6 292.5 ... 207.4 207.3
    TS       (time, lat, lon) float32 294.7 294.4 294.1 ... 282.7 282.5 282.4
    Z3       (time, plev, lat, lon) float32 122.5 122.5 ... 1.999e+04 1.999e+04
    PSL      (time, lat, lon) float32 1.014e+05 1.014e+05 ... 1.003e+05
    U        (time, plev, lat, lon) float32 -4.009 -3.789 -3.546 ... 12.22 12.25
    V        (time, plev, lat, lon) float32 5.606 5.629 5.695 ... -4.961 -5.141
    FLUT     (time, lat, lon) float32 274.2 274.7 275.2 ... 198.7 198.7 198.8
    TMQ      (time, lat, lon) float32 38.21 38.51 38.91 ... 12.82 12.83 12.82
    OMEGA    (time, plev, lat, lon) float32 0.008278 0.009047 ... -0.002001
    OCNFRAC  (time, lat, lon) float32 1.0 1.0 1.0 1.0 ... 0.9555 0.9496 0.9521
Attributes: (12/21)
    ne:                         0
    np:                         4
    Conventions:                CF-1.0
    source:                     CAM
    title:                      Regridded version of tmp.nc
    Version:                    $Name$
    ...                         ...
    history_of_appended_files:  Sun Apr 13 09:20:41 2025: Appended file /glad...
    remap_script:               ncremap
    remap_hostname:             casper-login1
    remap_version:              5.3.1
    map_file:                   /glade/work/zarzycki/maps/hyperion/map_ne0np4...
    input_file:                 /glade/u/home/zarzycki/scratch/hyperion/CHEY....

In [18]:
Control_DS_Dict['Sep-Nov']['T'].sel(plev=50000)

<xarray.DataArray 'T' (time: 270, lat: 61, lon: 101)>
array([[[268.09604, 268.10013, 268.09424, ..., 266.68442, 266.65347,
         266.62183],
        [268.07117, 268.06067, 268.0409 , ..., 266.6419 , 266.60287,
         266.5557 ],
        [268.01538, 267.98993, 267.94815, ..., 266.59323, 266.5505 ,
         266.49396],
        ...,
        [253.30377, 253.05507, 252.77951, ..., 252.84961, 252.8691 ,
         252.8873 ],
        [252.77164, 252.5132 , 252.23581, ..., 252.38657, 252.409  ,
         252.4276 ],
        [252.21738, 251.93982, 251.66875, ..., 251.96683, 252.00238,
         252.02615]],

       [[267.59756, 267.5808 , 267.58524, ..., 267.05093, 267.0444 ,
         267.03595],
        [267.54807, 267.5239 , 267.53754, ..., 267.02728, 267.0176 ,
         267.00626],
        [267.50156, 267.47574, 267.47495, ..., 266.99475, 266.9838 ,
         266.9724 ],
...
        [244.59575, 244.3292 , 244.09926, ..., 246.61205, 246.36702,
         246.14668],
        [244.4548 , 244.19724, 243.94713, ..., 246.06808, 245.84964,
         245.63911],
        [244.2893 , 244.03587, 243.80443, ..., 245.55223, 245.34378,
         245.14156]],

       [[266.72418, 266.72607, 266.72513, ..., 266.22653, 266.2042 ,
         266.18005],
        [266.675  , 266.67084, 266.66592, ..., 266.1651 , 266.15585,
         266.1423 ],
        [266.61465, 266.60202, 266.6082 , ..., 266.17752, 266.15714,
         266.15533],
        ...,
        [245.24263, 245.07559, 244.92169, ..., 247.75125, 247.75299,
         247.74492],
        [244.663  , 244.5053 , 244.35141, ..., 247.23227, 247.22937,
         247.21059],
        [244.1426 , 244.00734, 243.88135, ..., 246.65575, 246.64417,
         246.61452]]], dtype=float32)
Coordinates:
  * lat      (lat) float64 0.0 1.0 2.0 3.0 4.0 5.0 ... 56.0 57.0 58.0 59.0 60.0
  * lon      (lon) float64 -100.0 -99.0 -98.0 -97.0 -96.0 ... -3.0 -2.0 -1.0 0.0
    plev     float64 5e+04
  * time     (time) datetime64[ns] 1985-09-15 1986-09-15 ... 2014-11-27
Attributes:
    mdims:          1
    units:          K
    long_name:      Temperature
    cell_methods:   time: mean
    cell_measures:  area: area

In [19]:
# Define Constants and Variables
P0 = 100000
Rd = 287
cp = 1005
Temp = Control_DS_Dict['Sep-Nov']['T']
Plev = Control_DS_Dict['Sep-Nov']['plev']
Z = Control_DS_Dict['Sep-Nov']['Z3']
U = Control_DS_Dict['Sep-Nov']['U']
V = Control_DS_Dict['Sep-Nov']['V']

In [20]:
# Calculate Potential Temperature
Theta = Temp * (P0 / Plev) ** (Rd/cp)
#Control_DS_Dict['Sep-Nov']['THETA'] = Theta

In [21]:
# Calculate Coriolis Parameter
Lat = Control_DS_Dict['Sep-Nov']['lat']
f = 2 * 7.2921 * 10**-5 * numpy.sin(Lat / 180 * numpy.pi)
#Control_DS_Dict['Sep-Nov']['f'] = f

In [22]:
# Calculate Brunt Vaisala Frequency
dTheta = Theta.diff(dim='plev')
dZ = Z.diff(dim='plev')
Theta_Mid = (Theta.isel({'plev': slice(0, -1)}) + Theta.isel({'plev': slice(1, None)})) / 2
N2 = (dTheta / dZ) * 9.81 / Theta_Mid
N = numpy.sqrt(N2)
N
#Control_DS_Dict['Sep-Nov']['N'] = N

/glade/u/apps/opt/conda/envs/npl-2023b/lib/python3.10/site-packages/xarray/core/computation.py:761: RuntimeWarning: invalid value encountered in sqrt
  result_data = func(*input_data)


<xarray.DataArray (time: 270, plev: 11, lat: 61, lon: 101)>
array([[[[0.01042092, 0.01084826, 0.01118632, ..., 0.00746367,
          0.00753411, 0.00761512],
         [0.01024413, 0.01066109, 0.01111342, ..., 0.00728796,
          0.00739133, 0.00740736],
         [0.00967621, 0.01002621, 0.01045304, ..., 0.00727118,
          0.00734789, 0.00749086],
         ...,
         [0.01066256, 0.01074323, 0.01075699, ..., 0.00857854,
          0.0090521 , 0.0092455 ],
         [0.01080311, 0.01100938, 0.01131329, ..., 0.00819704,
          0.00835456, 0.00846086],
         [0.01122377, 0.01150027, 0.0117577 , ..., 0.00788925,
          0.00787968, 0.00784421]],

        [[0.01763867, 0.01781167, 0.01798672, ..., 0.01439846,
          0.01426735, 0.01408568],
         [0.01728105, 0.01746611, 0.01756756, ..., 0.01381355,
          0.01369442, 0.01362583],
         [0.01673092, 0.01694485, 0.01704812, ..., 0.0134575 ,
          0.0133637 , 0.01329556],
...
         [0.02215264, 0.02213914, 0.02212592, ..., 0.02061226,
          0.02063122, 0.02063598],
         [0.02209209, 0.02207205, 0.02204871, ..., 0.02072423,
          0.02073834, 0.02074024],
         [0.02203956, 0.0220072 , 0.0219799 , ..., 0.02077238,
          0.02078689, 0.02079377]],

        [[0.01478505, 0.01473707, 0.01467899, ..., 0.01467683,
          0.01467454, 0.0146793 ],
         [0.01483019, 0.01479068, 0.01474934, ..., 0.01467968,
          0.01467271, 0.01469543],
         [0.01492251, 0.01487262, 0.01487271, ..., 0.01475201,
          0.01475483, 0.01477068],
         ...,
         [0.02071882, 0.02068212, 0.02064865, ..., 0.02042004,
          0.02042489, 0.02042555],
         [0.02070752, 0.02067529, 0.02064485, ..., 0.020347  ,
          0.02034979, 0.02034701],
         [0.02069944, 0.02067429, 0.02065091, ..., 0.02028012,
          0.02028116, 0.0202749 ]]]])
Coordinates:
  * lat      (lat) float64 0.0 1.0 2.0 3.0 4.0 5.0 ... 56.0 57.0 58.0 59.0 60.0
  * lon      (lon) float64 -100.0 -99.0 -98.0 -97.0 -96.0 ... -3.0 -2.0 -1.0 0.0
  * plev     (plev) float64 9.25e+04 8.5e+04 7e+04 6e+04 ... 2e+04 1.5e+04 1e+04
  * time     (time) datetime64[ns] 1985-09-15 1986-09-15 ... 2014-11-27

In [30]:
# Calculate Vertical Gradient of Geostrophic Wind
Vg = numpy.sqrt(U**2 + V**2)
dU = Vg.diff(dim='plev')
dZ = Z.diff(dim='plev')
dU_dZ = dU / dZ
dU_dZ

<xarray.DataArray (time: 270, plev: 12, lat: 61, lon: 101)>
array([[[[ 1.37008146e-05, -1.74462271e-04, -4.89711470e-04, ...,
          -3.58762941e-03, -3.84697109e-03, -4.08359664e-03],
         [-1.12460461e-03, -1.36770005e-03, -1.65929296e-03, ...,
          -3.59076331e-03, -3.64172901e-03, -3.49710742e-03],
         [-2.21345341e-03, -2.48905667e-03, -2.87490501e-03, ...,
          -3.18314042e-03, -3.07254214e-03, -2.79360218e-03],
         ...,
         [ 3.38808843e-03,  3.52843036e-03,  3.74045921e-03, ...,
           2.66248919e-03,  1.84672303e-03,  1.50765083e-03],
         [ 3.71732539e-03,  3.89965880e-03,  4.06899629e-03, ...,
           2.29197741e-03,  1.88563520e-03,  1.67106022e-03],
         [ 4.01460379e-03,  4.31563472e-03,  4.52850433e-03, ...,
           2.46570073e-03,  2.26720236e-03,  2.13387492e-03]],

        [[ 1.54120848e-03,  1.51160790e-03,  1.60599907e-03, ...,
          -3.91210802e-03, -3.41905840e-03, -2.82445480e-03],
         [ 1.91551261e-03,  1.86376658e-03,  1.80548138e-03, ...,
          -3.81141948e-03, -3.68495332e-03, -3.75660183e-03],
         [ 1.84826378e-03,  1.80490047e-03,  1.86067051e-03, ...,
          -4.11860645e-03, -4.29903623e-03, -4.70632687e-03],
...
          -1.42238871e-03, -1.43465656e-03, -1.44178048e-03],
         [-1.20785774e-03, -1.18369155e-03, -1.17521966e-03, ...,
          -1.28357892e-03, -1.32021739e-03, -1.34055619e-03],
         [-1.29304209e-03, -1.29819149e-03, -1.30676723e-03, ...,
          -1.24292891e-03, -1.27786282e-03, -1.29453163e-03]],

        [[-7.02091260e-04, -6.07161433e-04, -5.55056613e-04, ...,
          -3.00117354e-05, -8.97241553e-05, -1.00920042e-04],
         [-5.73133584e-04, -5.13196224e-04, -4.84945747e-04, ...,
          -9.55575306e-05, -1.20078541e-04, -9.08704897e-05],
         [-4.08922555e-04, -3.90224392e-04, -4.11327201e-04, ...,
          -2.03739895e-04, -1.83503071e-04, -1.17122312e-04],
         ...,
         [-1.35475548e-03, -1.34927721e-03, -1.32942898e-03, ...,
          -7.76787696e-04, -7.09060521e-04, -6.46220113e-04],
         [-1.29584339e-03, -1.28389138e-03, -1.26507413e-03, ...,
          -6.92772737e-04, -6.29341404e-04, -5.65899594e-04],
         [-1.22773927e-03, -1.21453765e-03, -1.19719526e-03, ...,
          -5.80110063e-04, -5.26316639e-04, -4.74189001e-04]]]],
      dtype=float32)
Coordinates:
  * lat      (lat) float64 0.0 1.0 2.0 3.0 4.0 5.0 ... 56.0 57.0 58.0 59.0 60.0
  * lon      (lon) float64 -100.0 -99.0 -98.0 -97.0 -96.0 ... -3.0 -2.0 -1.0 0.0
  * plev     (plev) float64 9.25e+04 8.5e+04 7e+04 6e+04 ... 1.5e+04 1e+04 5e+03
  * time     (time) datetime64[ns] 1985-09-15 1986-09-15 ... 2014-11-27

In [32]:
# Calculate Eady Growth Rate
Sigma = 0.31 * numpy.abs(dU_dZ) * f / N
Sigma

<xarray.DataArray (time: 270, plev: 11, lat: 61, lon: 101)>
array([[[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [8.66212270e-08, 1.01225221e-07, 1.17808078e-07, ...,
          3.88759056e-07, 3.88762678e-07, 3.72516305e-07],
         [3.60934720e-07, 3.91707067e-07, 4.33954577e-07, ...,
          6.90739821e-07, 6.59779252e-07, 5.88431802e-07],
         ...,
         [1.21831073e-05, 1.25924914e-05, 1.33321180e-05, ...,
          1.18997831e-05, 7.82199035e-06, 6.25223372e-06],
         [1.33349592e-05, 1.37269346e-05, 1.39382544e-05, ...,
          1.08358597e-05, 8.74669095e-06, 7.65398311e-06],
         [1.40048588e-05, 1.46930338e-05, 1.50802025e-05, ...,
          1.22371295e-05, 1.12656662e-05, 1.06511119e-05]],

        [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [8.74610300e-08, 8.41966867e-08, 8.10926107e-08, ...,
          2.17711399e-07, 2.12318618e-07, 2.17536330e-07],
         [1.74304057e-07, 1.68065632e-07, 1.72209203e-07, ...,
          4.82891196e-07, 5.07584019e-07, 5.58520411e-07],
...
         [2.63920473e-06, 2.67483736e-06, 2.63291067e-06, ...,
          2.85884019e-06, 2.93569652e-06, 3.07897556e-06],
         [2.74223531e-06, 2.75400505e-06, 2.71668887e-06, ...,
          2.85122726e-06, 2.94347307e-06, 3.08006570e-06],
         [2.78146725e-06, 2.72817631e-06, 2.65283663e-06, ...,
          2.91073258e-06, 2.95585714e-06, 3.04784006e-06]],

        [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [1.66275846e-07, 1.57806528e-07, 1.44331343e-07, ...,
          1.74324908e-07, 1.62122117e-07, 1.51460894e-07],
         [2.61418772e-07, 2.38389115e-07, 2.11207711e-07, ...,
          3.30115378e-07, 3.14218976e-07, 2.97843030e-07],
         ...,
         [2.09373458e-06, 2.02278500e-06, 2.01989194e-06, ...,
          2.67070844e-06, 2.69310382e-06, 2.70638822e-06],
         [2.26046394e-06, 2.21869076e-06, 2.20605985e-06, ...,
          2.44473656e-06, 2.51417500e-06, 2.55325586e-06],
         [2.44584520e-06, 2.45857249e-06, 2.47761607e-06, ...,
          2.39966592e-06, 2.46698420e-06, 2.49993584e-06]]]])
Coordinates:
  * lat      (lat) float64 0.0 1.0 2.0 3.0 4.0 5.0 ... 56.0 57.0 58.0 59.0 60.0
  * lon      (lon) float64 -100.0 -99.0 -98.0 -97.0 -96.0 ... -3.0 -2.0 -1.0 0.0
  * plev     (plev) float64 9.25e+04 8.5e+04 7e+04 6e+04 ... 2e+04 1.5e+04 1e+04
  * time     (time) datetime64[ns] 1985-09-15 1986-09-15 ... 2014-11-27

In [31]:
dU_dZ.sel(plev=70000, lat=28, lon=-72)

<xarray.DataArray (time: 270)>
array([-1.6791797e-04, -1.8196063e-04,  4.3813656e-05, -4.9061998e-04,
       -1.6868299e-04,  9.0327529e-05, -9.6189760e-04, -5.9121932e-05,
       -7.5477653e-04,  1.9982118e-04, -9.1810426e-04, -6.1474735e-04,
        2.7656241e-04,  5.9731153e-04, -1.7968076e-04, -2.6754965e-04,
        2.8699407e-04, -2.7703209e-04, -1.3809308e-04,  8.8861899e-04,
       -6.6066749e-04,  7.9055887e-04,  9.1134442e-04,  7.1937032e-04,
       -4.7530787e-04, -9.7250333e-04,  4.2680532e-04, -1.0813335e-03,
        7.4073905e-04, -1.3113867e-03,  2.2503347e-04,  2.2929792e-04,
       -8.3893567e-04, -9.9123374e-04,  1.2062847e-03, -4.6899868e-04,
       -3.0825453e-04, -3.8815325e-04,  1.9966322e-04,  3.3610268e-04,
        7.0388691e-04, -3.6462207e-04,  9.1779482e-04, -1.4773895e-03,
       -1.0984634e-03, -1.7699967e-04,  1.0331301e-03, -8.8696781e-04,
       -1.3902555e-04, -6.6584779e-04,  5.2747381e-04, -8.2950626e-04,
       -2.3313658e-04, -1.2860155e-04, -8.4479054e-04, -2.3794705e-04,
       -4.3918885e-04,  8.0367585e-04,  4.4133142e-04,  4.6968235e-05,
        2.3762704e-04, -1.7599476e-04,  9.9929515e-04,  9.0996281e-04,
       -1.6827024e-04, -1.5519882e-04,  3.4709435e-04,  5.5129756e-04,
       -4.3762955e-04,  1.1616932e-03, -5.7919731e-04, -5.2851508e-04,
        9.7134201e-05, -7.4027880e-04, -9.1554728e-05,  6.4834989e-05,
       -2.7013241e-04, -7.4900303e-04, -9.1546401e-04, -2.0691202e-04,
...
        1.4190895e-03,  3.6641669e-03,  3.4317402e-03,  1.0245716e-03,
        2.0497898e-03,  2.7357377e-03, -1.5792160e-03,  3.5217882e-04,
        3.2585601e-03, -1.4616745e-03, -6.2154641e-04,  2.3439750e-03,
        8.1197091e-04,  2.0400726e-04,  3.0924138e-03,  1.3323929e-03,
       -3.8476166e-04,  3.0540067e-03,  3.2774606e-03,  3.0522833e-03,
        4.7456939e-03,  1.6366986e-03,  2.5991891e-03,  1.6562565e-03,
        3.4900852e-03,  2.4656923e-03,  1.2279813e-04,  2.7039165e-03,
        2.3551099e-03,  2.3948192e-03,  3.1818210e-03,  2.5576062e-03,
        2.7969866e-03,  3.0420085e-03,  1.1481410e-03,  2.1513337e-03,
        2.6410241e-03,  2.5321965e-03, -1.3693894e-03,  3.7709752e-03,
        2.4215903e-03,  2.7391813e-03, -1.4306369e-04,  3.5824392e-03,
       -2.3556130e-03,  1.9111617e-03,  2.9800504e-03,  1.9968709e-03,
        2.3303591e-03,  3.0922501e-03,  2.7934788e-03,  2.2726243e-03,
        2.8260422e-03,  3.0227094e-03,  2.4437900e-03,  2.7446658e-03,
        3.1399548e-03,  3.2836546e-03,  2.2710236e-03,  1.5050590e-03,
        3.4396451e-03,  3.3826365e-03,  3.4503269e-03,  3.3873846e-03,
        3.9032763e-03,  2.6002345e-03,  3.0334312e-03,  1.9242929e-03,
        3.2822683e-03,  7.3926389e-04,  2.3980935e-03,  3.7994268e-03,
        3.5163069e-03,  2.8155029e-03,  1.5175875e-03,  1.9996283e-03,
        3.2846383e-03,  1.7004536e-03], dtype=float32)
Coordinates:
    lat      float64 28.0
    lon      float64 -72.0
    plev     float64 7e+04
  * time     (time) datetime64[ns] 1985-09-15 1986-09-15 ... 2014-11-27

In [29]:
dU_dZ.sel(plev=70000, lat=28, lon=-72)

<xarray.DataArray (time: 270)>
array([5.9481780e-04, 6.3031435e-04, 6.4706203e-04, 1.2622555e-03,
       5.5859482e-04, 1.2358306e-03, 9.7472029e-04, 4.8562637e-05,
       5.5461180e-05, 7.1472814e-04, 6.5263460e-04, 3.6163916e-04,
       1.2885226e-03, 1.7513802e-03, 1.1411998e-03, 5.3531764e-04,
       1.4664410e-03, 8.0040493e-04, 1.2584467e-03, 1.8206971e-03,
       1.0833670e-03, 1.1311079e-03, 9.3025545e-04, 8.5287535e-04,
       4.1490412e-04, 9.6297177e-04, 7.0348429e-04, 1.0974080e-03,
       1.4355748e-03, 9.8734698e-04, 2.2438231e-04, 1.0384825e-03,
       6.9209276e-04, 1.2660477e-03, 1.2099342e-03, 1.2545136e-03,
       7.1237946e-04, 7.7587477e-04, 1.3475415e-03, 5.9019664e-04,
       1.2863177e-03, 1.1964638e-03, 1.0357624e-03, 1.6702020e-03,
       1.0868700e-03, 1.6326687e-03, 1.1544765e-03, 8.1390457e-04,
       9.1532950e-04, 6.5588910e-04, 9.1512169e-04, 8.0543896e-04,
       1.0418033e-03, 1.3481780e-03, 8.3690113e-04, 1.0418417e-03,
       1.0619570e-03, 1.0861529e-03, 9.6920639e-04, 8.5667375e-04,
       1.0443949e-03, 1.0828277e-03, 1.2318617e-03, 1.8793362e-03,
       5.5437483e-04, 1.2377513e-03, 1.4107331e-03, 2.1626945e-03,
       6.2191545e-04, 1.3428859e-03, 8.3454885e-04, 1.3383857e-03,
       1.6459586e-03, 5.5920205e-04, 7.0873997e-04, 8.7428140e-04,
       9.4327122e-05, 8.8751136e-04, 1.1562922e-03, 6.3794712e-04,
...
       2.0697012e-03, 3.6569270e-03, 3.5020947e-03, 2.2738762e-03,
       2.2166772e-03, 2.7286361e-03, 2.0871095e-03, 2.3998781e-03,
       3.2744228e-03, 1.4359942e-03, 2.6354499e-03, 2.3810670e-03,
       1.6498959e-03, 2.1933438e-03, 3.1686313e-03, 2.4765725e-03,
       2.2211359e-03, 3.2920681e-03, 3.2669816e-03, 3.5975599e-03,
       4.6695354e-03, 2.1096827e-03, 2.6708192e-03, 2.2159093e-03,
       3.4970199e-03, 2.5054540e-03, 3.0140397e-03, 2.8854369e-03,
       2.3711452e-03, 2.3927686e-03, 3.1767206e-03, 2.5602751e-03,
       2.7474049e-03, 3.1166766e-03, 2.4260106e-03, 2.3728379e-03,
       2.7126349e-03, 2.6190698e-03, 2.4907861e-03, 3.5734798e-03,
       2.8315522e-03, 2.7180931e-03, 2.6797701e-03, 3.9235167e-03,
       2.7251383e-03, 2.4242816e-03, 3.0423701e-03, 2.9417896e-03,
       2.7048599e-03, 3.3942971e-03, 2.8006912e-03, 2.3375843e-03,
       2.7150416e-03, 3.0726972e-03, 2.4968991e-03, 2.9734622e-03,
       3.5916078e-03, 3.2939841e-03, 2.8272234e-03, 2.8001505e-03,
       3.3498229e-03, 3.3695572e-03, 3.2502776e-03, 3.7973248e-03,
       3.8187650e-03, 2.6351190e-03, 3.0263462e-03, 2.0025792e-03,
       3.5426803e-03, 3.0751436e-03, 2.8489423e-03, 3.8539288e-03,
       3.4813343e-03, 2.8500366e-03, 2.6719861e-03, 2.0098928e-03,
       3.1175397e-03, 3.3249022e-03], dtype=float32)
Coordinates:
    lat      float64 28.0
    lon      float64 -72.0
    plev     float64 7e+04
  * time     (time) datetime64[ns] 1985-09-15 1986-09-15 ... 2014-11-27

In [33]:
Sigma.sel(plev=70000, lat=28, lon=-72)

<xarray.DataArray (time: 270)>
array([2.93438021e-07, 3.04120709e-07, 7.47657090e-08, 8.61071197e-07,
       2.92357815e-07, 1.52121825e-07, 1.63556066e-06, 1.00204900e-07,
       1.33031170e-06, 3.41685864e-07, 1.56176108e-06, 1.09509274e-06,
       4.78503042e-07, 1.05691473e-06, 3.02007958e-07, 4.64261137e-07,
       4.73597487e-07, 4.73460655e-07, 2.37697055e-07, 1.57084455e-06,
       1.12681125e-06, 1.32195170e-06, 1.61560345e-06, 1.22165257e-06,
       8.18751220e-07, 1.62679607e-06, 7.11609798e-07, 1.84008589e-06,
       1.37717502e-06, 2.24780455e-06, 3.86855010e-07, 4.28278746e-07,
       1.45663583e-06, 1.64577734e-06, 2.08502896e-06, 7.99662054e-07,
       5.12903569e-07, 6.68379552e-07, 3.61477826e-07, 5.78649272e-07,
       1.23158256e-06, 6.32047217e-07, 1.61998191e-06, 2.58705690e-06,
       1.92496968e-06, 2.94673170e-07, 1.74456712e-06, 1.52723363e-06,
       2.42290914e-07, 1.11662226e-06, 9.00254618e-07, 1.42023733e-06,
       4.07353452e-07, 2.15309016e-07, 1.45362577e-06, 4.19194550e-07,
       7.48490306e-07, 1.38336177e-06, 7.87555362e-07, 8.11951070e-08,
       3.95193280e-07, 3.08452664e-07, 1.71164716e-06, 1.54623496e-06,
       2.90054817e-07, 2.63001911e-07, 5.89000120e-07, 9.67132946e-07,
       7.53748761e-07, 2.03249689e-06, 1.00080592e-06, 9.21906361e-07,
       1.71285899e-07, 1.23007714e-06, 1.58798961e-07, 1.10848189e-07,
       4.45694113e-07, 1.25093357e-06, 1.53308288e-06, 3.63437403e-07,
...
       2.32382052e-06, 5.30193121e-06, 4.99425832e-06, 1.56047684e-06,
       3.17867466e-06, 4.20117039e-06, 2.46242463e-06, 5.33488180e-07,
       4.91918502e-06, 2.44710885e-06, 9.69427335e-07, 3.44354381e-06,
       1.27973098e-06, 3.00120682e-07, 4.47377433e-06, 1.92832522e-06,
       5.93635846e-07, 4.98370343e-06, 4.94832883e-06, 4.90850080e-06,
       7.06063789e-06, 2.50301775e-06, 3.99820878e-06, 2.38982981e-06,
       5.46191231e-06, 3.75750275e-06, 1.78300117e-07, 3.98487642e-06,
       3.55569660e-06, 3.57740239e-06, 5.31231159e-06, 4.17381369e-06,
       4.23988034e-06, 4.54546744e-06, 1.61045594e-06, 3.36962940e-06,
       3.96039348e-06, 3.87553678e-06, 2.13691149e-06, 5.47061660e-06,
       3.55551066e-06, 3.98114173e-06, 2.15259809e-07, 5.20708124e-06,
       3.82320324e-06, 2.95861975e-06, 4.54787946e-06, 3.21412049e-06,
       3.48242038e-06, 4.86426973e-06, 4.33130361e-06, 3.40597221e-06,
       4.44502479e-06, 4.38825417e-06, 3.57170126e-06, 4.22237304e-06,
       5.08885859e-06, 4.77565784e-06, 3.42052394e-06, 2.36045370e-06,
       5.17450547e-06, 5.09733622e-06, 4.88020500e-06, 5.03417705e-06,
       6.13773261e-06, 4.20789031e-06, 4.51025307e-06, 2.75623144e-06,
       4.80882395e-06, 1.15563976e-06, 3.72232676e-06, 5.22609565e-06,
       5.44420705e-06, 4.16247604e-06, 2.33717900e-06, 3.16793999e-06,
       5.03167689e-06, 2.57084540e-06])
Coordinates:
    lat      float64 28.0
    lon      float64 -72.0
    plev     float64 7e+04
  * time     (time) datetime64[ns] 1985-09-15 1986-09-15 ... 2014-11-27

In [27]:
Sigma.sel(plev=70000, lat=28, lon=-72)

<xarray.DataArray (time: 270)>
array([1.03944891e-06, 1.05347871e-06, 1.10417743e-06, 2.21534359e-06,
       9.68144807e-07, 2.08127920e-06, 1.65736367e-06, 8.23081044e-08,
       9.77516580e-08, 1.22215532e-06, 1.11017820e-06, 6.44213329e-07,
       2.22937741e-06, 3.09898491e-06, 1.91813216e-06, 9.28901121e-07,
       2.41992029e-06, 1.36792903e-06, 2.16614105e-06, 3.21851345e-06,
       1.84775276e-06, 1.89140872e-06, 1.64912848e-06, 1.44837416e-06,
       7.14701468e-07, 1.61085169e-06, 1.17291484e-06, 1.86743961e-06,
       2.66900722e-06, 1.69237881e-06, 3.85735616e-07, 1.93965982e-06,
       1.20167392e-06, 2.10205980e-06, 2.09133706e-06, 2.13899735e-06,
       1.18532556e-06, 1.33601570e-06, 2.43963991e-06, 1.01610868e-06,
       2.25065482e-06, 2.07398747e-06, 1.82820422e-06, 2.92469090e-06,
       1.90465306e-06, 2.71810486e-06, 1.94947550e-06, 1.40142910e-06,
       1.59521767e-06, 1.09992155e-06, 1.56186426e-06, 1.37903045e-06,
       1.82031554e-06, 2.25716472e-06, 1.44005054e-06, 1.83542675e-06,
       1.80984669e-06, 1.86958747e-06, 1.72954754e-06, 1.48095242e-06,
       1.73691441e-06, 1.89779002e-06, 2.10999982e-06, 3.19342227e-06,
       9.55600299e-07, 2.09750922e-06, 2.39393685e-06, 3.79398218e-06,
       1.07115249e-06, 2.34951130e-06, 1.44203269e-06, 2.33459052e-06,
       2.90247411e-06, 9.29192685e-07, 1.22928849e-06, 1.49475631e-06,
       1.55631243e-07, 1.48226068e-06, 1.93638610e-06, 1.12054319e-06,
...
       3.38922536e-06, 5.29145530e-06, 5.09664587e-06, 3.46323397e-06,
       3.43747256e-06, 4.19026485e-06, 3.25436767e-06, 3.63538809e-06,
       4.94313139e-06, 2.40411539e-06, 4.11051698e-06, 3.49803581e-06,
       2.60036761e-06, 3.22668834e-06, 4.58403766e-06, 3.58425618e-06,
       3.42691601e-06, 5.37218607e-06, 4.93250775e-06, 5.78538193e-06,
       6.94732932e-06, 3.22635640e-06, 4.10839404e-06, 3.19735861e-06,
       5.47276482e-06, 3.81809609e-06, 4.37631779e-06, 4.25239073e-06,
       3.57990632e-06, 3.57433919e-06, 5.30379590e-06, 4.17816915e-06,
       4.16472048e-06, 4.65703883e-06, 3.40287742e-06, 3.71657093e-06,
       4.06777873e-06, 4.00849692e-06, 3.88683425e-06, 5.18410699e-06,
       4.15743920e-06, 3.95049194e-06, 4.03209782e-06, 5.70283774e-06,
       4.42294930e-06, 3.75296695e-06, 4.64298586e-06, 4.73504142e-06,
       4.04206363e-06, 5.33940543e-06, 4.34248661e-06, 3.50332767e-06,
       4.27043433e-06, 4.46082455e-06, 3.64932235e-06, 4.57435180e-06,
       5.82084332e-06, 4.79068121e-06, 4.25824980e-06, 4.39160575e-06,
       5.03937988e-06, 5.07762720e-06, 4.59725175e-06, 5.64341127e-06,
       6.00484251e-06, 4.26434314e-06, 4.49971875e-06, 2.86836363e-06,
       5.19035148e-06, 4.80715770e-06, 4.42213534e-06, 5.30106278e-06,
       5.39006035e-06, 4.21353106e-06, 4.11502446e-06, 3.18420189e-06,
       4.77570161e-06, 5.02678208e-06])
Coordinates:
    lat      float64 28.0
    lon      float64 -72.0
    plev     float64 7e+04
  * time     (time) datetime64[ns] 1985-09-15 1986-09-15 ... 2014-11-27